### Import libraries

In [1]:
from langchain.globals import set_llm_cache
from langchain_community.cache import AzureCosmosDBSemanticCache
from langchain_community.vectorstores.azure_cosmos_db import (
    CosmosDBSimilarityType,
    CosmosDBVectorSearchType,
)
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores.azure_cosmos_db import AzureCosmosDBVectorSearch

from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import AzureOpenAI
import os, time
from pymongo import MongoClient

load_dotenv()

True

### Helper functions

In [2]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
def generate_embeddings(client, text):
    return client.embeddings.create(input = [text], model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")).data[0].embedding

In [3]:
def get_similar_docs(collection, query_vector, limit=5):
    pipeline = [
            {
                '$search': {
                    'cosmosSearch': {
                        'vector': query_vector,
                        'path': 'embedding',
                        'k': limit
                    },
                    'returnStoredSource': True
                }
            }
        ]

    docs = []
    for doc in collection.aggregate(pipeline):
        serialized_doc = {**doc, "_id": str(doc["_id"])}
        docs.append(serialized_doc)

    return docs

In [4]:
def generate_completion(client, context, query):
    result = ""
    try:
        
        prompt = f"""
        Only use the following CONTEXT to answer the QUESTION. Do not use any other knowledge or information.

        CONTEXT:
        {context}

        QUESTION:
        {query}
        """

        response = client.chat.completions.create(
            model=os.getenv("AZURE_OPENAI_CHAT_MODEL"), 
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        
        result = response.choices[0].message.content.replace('\n',' ')

    except Exception as e:        
        print(f"An unexpected error occurred: {e}")

    return result

### Init semantic cache

In [5]:

# Init Cosmos DB for MongoDB
CONNECTION_STRING = os.getenv("AZURE_COSMOSDB_MONGODB_CONNECTION_STRING")
DATABASE_NAME = os.getenv("AZURE_COSMOSDB_MONGODB_DATABASE")
COLLECTION_NAME = os.getenv("AZURE_COSMOSDB_MONGODB_COLLECTION")

client = MongoClient(CONNECTION_STRING)   
database = client[DATABASE_NAME]
collection = database[COLLECTION_NAME]

# Init OpenAI client
openai_client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_KEY"),  
    api_version = os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Init Semantic Cache
num_lists = 3
dimensions = 1536
similarity_algorithm = CosmosDBSimilarityType.COS
kind = CosmosDBVectorSearchType.VECTOR_IVF
m = 16
ef_construction = 64
ef_search = 40
score_threshold = 0.9
application_name = "Semantic Caching Application"

vector_store = AzureCosmosDBVectorSearch.from_connection_string(
    CONNECTION_STRING, 
    DATABASE_NAME + "." + COLLECTION_NAME, 
    AzureOpenAIEmbeddings(), 
    index_name="vectorindex"
)

semantic_cache = AzureCosmosDBSemanticCache(
        cosmosdb_connection_string=CONNECTION_STRING,
        cosmosdb_client=None,
        embedding=AzureOpenAIEmbeddings(),
        database_name=DATABASE_NAME,
        collection_name=COLLECTION_NAME,
        num_lists=num_lists,
        similarity=similarity_algorithm,
        kind=kind,
        dimensions=dimensions,
        m=m,
        ef_construction=ef_construction,
        ef_search=ef_search,
        score_threshold=score_threshold,
        application_name=application_name,
    )

set_llm_cache(semantic_cache)

C:\Users\pattruong\AppData\Local\Temp\ipykernel_10760\3999930758.py:6: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client = MongoClient(CONNECTION_STRING)
c:\data\00_dev\semantic-cache\.venv\Lib\site-packages\langchain_community\vectorstores\azure_cosmos_db.py:145: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client: MongoClient = MongoClient(connection_string, appname=appname)


### Queries

In [6]:
# The first time, it is not yet in cache, so it should take longer
query = "Qu'est-ce qu'un plastique à usage unique?"
vector = generate_embeddings(openai_client, query)

start_time = time.time()
docs = get_similar_docs(collection, vector)
print(docs)
end_time = time.time()
print(f"Docs retrieved in {end_time - start_time:.3f} seconds")

[{'_id': '662a6b689b483fe1e6b0fac5', 'id': '4c16ee7c-cdf2-48ef-a5f7-73c1824d38bb', 'file_uri': 'https://sttsdatafrkb.blob.core.windows.net/raw/pdf/paris_journal.pdf', 'page': '11', 'doc_type': 'pdf', 'content': "400 000.bouteilles en plastique.étaient jusque-là.consommées par les.quelque 50 000 coureurs.du Marathon de Paris.chaque année..ZÉRO.PLASTIQUE.À USAGE UNIQUE.-.@ Getty Images/master1305.Auprès des acteurs.du tourisme.La Ville de Paris a lancé en.avril 2023 la première certification.zéro plastique pour les acteurs.économiques. Déjà plus de cent.entreprises se sont engagées pour.2024. Cela concerne les musées,.les croisiéristes sur la Seine, les bars,.les aéroports ... Tous les.établissements engagés devront.garantir un accès libre à l'eau du.robinet et permettre de remplir.librement sa gourde..Plus de 200 entreprises.et acteurs économiques sont.déjà certifiés zéro plastique.par la Ville de Paris. ».Suivez l'actualité des Jeux olympiques.et paralympiques sur.Paris.fr/jeux2024.11 

In [7]:
print(generate_completion(openai_client, docs, query))

        Un plastique à usage unique (PUU) est défini, selon la réglementation européenne, comme des emballages contenant totalement ou partiellement du plastique. Cela inclut les canettes en aluminium/acier, les briques, les gobelets en carton ainsi que le plastique biosourcé, compostable ou recyclé, tous étant considérés comme des contenants en PUU.


In [8]:
# The second time, it is in cache, so it should be quicker
query = "Tell me about single use plastic"
vector = generate_embeddings(openai_client, query)

start_time = time.time()
docs = get_similar_docs(collection, vector)
print(docs)
end_time = time.time()
print(f"Docs retrieved in {end_time - start_time:.3f} seconds")

[{'_id': '662a6b689b483fe1e6b0fac5', 'id': '4c16ee7c-cdf2-48ef-a5f7-73c1824d38bb', 'file_uri': 'https://sttsdatafrkb.blob.core.windows.net/raw/pdf/paris_journal.pdf', 'page': '11', 'doc_type': 'pdf', 'content': "400 000.bouteilles en plastique.étaient jusque-là.consommées par les.quelque 50 000 coureurs.du Marathon de Paris.chaque année..ZÉRO.PLASTIQUE.À USAGE UNIQUE.-.@ Getty Images/master1305.Auprès des acteurs.du tourisme.La Ville de Paris a lancé en.avril 2023 la première certification.zéro plastique pour les acteurs.économiques. Déjà plus de cent.entreprises se sont engagées pour.2024. Cela concerne les musées,.les croisiéristes sur la Seine, les bars,.les aéroports ... Tous les.établissements engagés devront.garantir un accès libre à l'eau du.robinet et permettre de remplir.librement sa gourde..Plus de 200 entreprises.et acteurs économiques sont.déjà certifiés zéro plastique.par la Ville de Paris. ».Suivez l'actualité des Jeux olympiques.et paralympiques sur.Paris.fr/jeux2024.11 

In [9]:
print(generate_completion(openai_client, docs, query))

Single-use plastics, often abbreviated as SUPs or PUUs (plastique à usage unique), refer to disposable plastic items designed to be used once before being discarded. These include products like bottles, bags, straws, and packaging materials. Efforts have been made to reduce and eventually eliminate the use of these plastics due to their significant environmental impact.  **Paris Initiatives:** - **Paris Marathon and Events:** Paris has initiated steps to eliminate single-use plastics in city events, such as the Marathon for All during the 2024 Olympics, where alternatives like reusable cups and water points will replace plastic bottles. - **Tourist and Public Spaces:** Public water access points and reusable options will be promoted in various public and tourist spots like museums, river cruises, and airports. - **City Regulations:** From September 2024, the use of single-use plastic packaging will be prohibited at public events, mandating the use of alternative solutions like water di